In [ ]:
import boto3
import pandas as pd

def get_aws_client(service,access_key,secret_key,region):
    s3_client = boto3.client(service_name = service, 
                            aws_access_key_id = access_key,
                            aws_secret_access_key = secret_key,
                            region_name = region)
    return s3_client


def s3_list_bucket_names(s3_client):
    response = s3_client.list_buckets()
    buckets = []
    for bucket in response['Buckets']:
        buckets.append(bucket["Name"])
    return buckets

def s3_get_data_structure(s3_client,bucket_name):
    try:
        res = s3_client.list_objects(Bucket = bucket_name)['Contents']
    except Exception as e:
        return []
    paths = []
    for i in res:
        paths.append(i['Key'])
    return paths

def s3_create_folder(s3_client,bucket_name,folder_path):
    s3_client.put_object(Bucket=bucket_name, Key=(folder_path+'/'))
    
def s3_check_folder_exists(s3_client,bucket_name,folder_path):
    path = path.rstrip('/') 
    resp = s3_client.list_objects(Bucket=bucket_name, Prefix=folder_path, Delimiter='/',MaxKeys=1)
    return 'CommonPrefixes' in resp

def upload_file(s3_client,local_file_path, bucket_name, s3_file_path):
    s3_client.upload_file(Filename=local_file_path,
                          Bucket=bucket_name,
                          Key=s3_file_path)
    
def get_file(s3_client,bucket_name, file_path):
    obj = s3_client.get_object(Bucket= bucket_name, Key= file_path) 
    return obj['Body']

def generate_presigned_url(s3_client, bucket_name, object_key, expiration_time=604800):
    url = s3_client.generate_presigned_url(
        'get_object',
        Params={
            'Bucket': bucket_name,
            'Key': object_key
        },
        ExpiresIn=expiration_time
    )

    return url

def get_df_from_s3(s3_client, bucket_name, s3_key):
    obj = s3_client.get_object(Bucket = bucket_name, Key = s3_key) 
    return pd.read_csv(obj['Body'])

def delete_object(s3_client, bucket_name,s3_key):
    s3_client.delete_object(Bucket=bucket_name, Key=s3_key)


**Note:** The 's3_key' or 's3_file_path' must begin with the folder inside the specified bucket. For example, if you intend to upload the file 'data.csv' to the bucket 'sample-bucket' within the folder 'sample_data/data/my_data.csv,' follow these steps:
1. Call the function 'get_aws_client' to obtain the 's3_client.'
2. Utilize the 'upload_file' function with the following parameters:
* It's important to note that the full S3 file path, "s3://sample-bucket/sample_data/data/my_data.csv," should not be used directly for the s3_file_path parameter. Instead, only the file path after the bucket name should be specified. The necessary folders in S3 will be automatically created when the file is uploaded to the specified path.
    - `local_file_path` set to 'data.csv'
    - `bucket_name` set to 'sample-bucket'
    - `s3_file_path` set to 'sample_data/data/my_data.csv'

*Note:* There is no need to manually create folders in S3; they will be automatically generated once the file is uploaded to the specified path.
